# Steam インディーゲーム市場 - 探索的データ分析

## 概要
Steam APIから収集したインディーゲームデータを用いて、市場トレンドと成功要因を分析します。

### 分析目標
1. インディーゲーム市場の全体像把握
2. ジャンル別・価格帯別の傾向分析  
3. 成功ゲームの特徴抽出
4. 開発者・パブリッシャー分析

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import warnings

# 警告を非表示
warnings.filterwarnings('ignore')

# 環境変数読み込み
load_dotenv()

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. データベース接続とデータ読み込み

In [ ]:
# データベース接続設定
DB_CONFIG = {
    "host": os.getenv("POSTGRES_HOST", "postgres"),
    "port": int(os.getenv("POSTGRES_PORT", 5432)),
    "database": os.getenv("POSTGRES_DB", "steam_analytics"),
    "user": os.getenv("POSTGRES_USER", "steam_user"),
    "password": os.getenv("POSTGRES_PASSWORD", "steam_password"),
}

# SQLAlchemy エンジン作成
engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

print("✅ データベース接続設定完了")

In [ ]:
# ゲームデータの読み込み
query = """
SELECT 
    app_id,
    name,
    type,
    is_free,
    short_description,
    developers,
    publishers,
    price_currency,
    price_initial,
    price_final,
    price_discount_percent,
    release_date_text,
    release_date_coming_soon,
    platforms_windows,
    platforms_mac,
    platforms_linux,
    genres,
    categories,
    positive_reviews,
    negative_reviews,
    total_reviews,
    created_at
FROM games
WHERE type = 'game'  -- ゲームのみに絞る
ORDER BY created_at DESC;
"""

df = pd.read_sql_query(query, engine)

print(f"📊 データ読み込み完了: {len(df):,}件のゲーム")
print(f"📅 データ期間: {df['created_at'].min()} ～ {df['created_at'].max()}")

# データの基本情報
df.info()

## 2. データ前処理とクリーニング

In [ ]:
# データ前処理
def preprocess_data(df):
    """データの前処理を実行"""
    df_clean = df.copy()
    
    # 価格データの変換（セント → ドル）
    df_clean['price_usd'] = df_clean['price_final'] / 100
    df_clean.loc[df_clean['is_free'] == True, 'price_usd'] = 0
    
    # インディーゲーム判定
    def is_indie_game(row):
        """インディーゲーム判定ロジック"""
        if row['genres'] is None:
            return False
            
        # ジャンルにIndieが含まれる
        if any('Indie' in str(genre) for genre in row['genres'] if genre):
            return True
            
        # 開発者とパブリッシャーが同じ（セルフパブリッシング）
        if (row['developers'] is not None and row['publishers'] is not None and 
            len(row['developers']) <= 2 and set(row['developers']) == set(row['publishers'])):
            return True
            
        return False
    
    df_clean['is_indie'] = df_clean.apply(is_indie_game, axis=1)
    
    # ジャンルデータの展開
    all_genres = []
    for genres in df_clean['genres'].dropna():
        if isinstance(genres, list):
            all_genres.extend(genres)
    
    df_clean['primary_genre'] = df_clean['genres'].apply(
        lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'Other'
    )
    
    # 開発者データの処理
    df_clean['primary_developer'] = df_clean['developers'].apply(
        lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 'Unknown'
    )
    
    # プラットフォーム数の計算
    df_clean['platform_count'] = (
        df_clean['platforms_windows'].astype(int) + 
        df_clean['platforms_mac'].astype(int) + 
        df_clean['platforms_linux'].astype(int)
    )
    
    # 価格帯カテゴリ
    def price_category(price):
        if price == 0:
            return 'Free'
        elif price < 5:
            return 'Budget ($0-5)'
        elif price < 15:
            return 'Mid-range ($5-15)'
        elif price < 30:
            return 'Premium ($15-30)'
        else:
            return 'AAA ($30+)'
    
    df_clean['price_category'] = df_clean['price_usd'].apply(price_category)
    
    return df_clean

# 前処理実行
df_processed = preprocess_data(df)

print(f"✅ データ前処理完了")
print(f"🎯 インディーゲーム: {df_processed['is_indie'].sum():,}件 ({df_processed['is_indie'].mean()*100:.1f}%)")
print(f"💰 有料ゲーム: {(df_processed['price_usd'] > 0).sum():,}件")
print(f"🆓 無料ゲーム: {(df_processed['price_usd'] == 0).sum():,}件")

## 3. 基本統計とデータ概要

In [ ]:
# 基本統計サマリー
print("📊 インディーゲーム市場 基本統計\n" + "="*50)

# 全体統計
total_games = len(df_processed)
indie_games = df_processed['is_indie'].sum()
indie_ratio = indie_games / total_games * 100

print(f"🎮 総ゲーム数: {total_games:,}件")
print(f"🎯 インディーゲーム: {indie_games:,}件 ({indie_ratio:.1f}%)")
print(f"🏢 非インディー: {total_games - indie_games:,}件 ({100-indie_ratio:.1f}%)")

# 価格統計（有料ゲームのみ）
paid_games = df_processed[df_processed['price_usd'] > 0]
print(f"\n💰 価格統計 (有料ゲーム {len(paid_games):,}件)")
print(f"   平均価格: ${paid_games['price_usd'].mean():.2f}")
print(f"   中央値: ${paid_games['price_usd'].median():.2f}")
print(f"   最高価格: ${paid_games['price_usd'].max():.2f}")
print(f"   最低価格: ${paid_games['price_usd'].min():.2f}")

# インディーゲーム vs 非インディーゲームの価格比較
indie_paid = paid_games[paid_games['is_indie'] == True]
non_indie_paid = paid_games[paid_games['is_indie'] == False]

if len(indie_paid) > 0 and len(non_indie_paid) > 0:
    print(f"\n🎯 インディー vs 非インディー価格比較")
    print(f"   インディー平均: ${indie_paid['price_usd'].mean():.2f}")
    print(f"   非インディー平均: ${non_indie_paid['price_usd'].mean():.2f}")

# プラットフォーム統計
print(f"\n🖥️ プラットフォーム対応率")
print(f"   Windows: {df_processed['platforms_windows'].mean()*100:.1f}%")
print(f"   Mac: {df_processed['platforms_mac'].mean()*100:.1f}%")
print(f"   Linux: {df_processed['platforms_linux'].mean()*100:.1f}%")
print(f"   平均対応数: {df_processed['platform_count'].mean():.1f}")

## 4. ジャンル分析

In [ ]:
# ジャンル別統計
genre_stats = df_processed.groupby('primary_genre').agg({
    'app_id': 'count',
    'is_indie': 'sum',
    'price_usd': 'mean',
    'platform_count': 'mean'
}).round(2)

genre_stats.columns = ['ゲーム数', 'インディー数', '平均価格', '平均プラットフォーム数']
genre_stats['インディー率'] = (genre_stats['インディー数'] / genre_stats['ゲーム数'] * 100).round(1)
genre_stats = genre_stats.sort_values('ゲーム数', ascending=False)

print("🎮 ジャンル別分析 (上位10ジャンル)")
print("="*80)
print(genre_stats.head(10))

In [ ]:
# ジャンル分布の可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. ジャンル別ゲーム数（上位10）
top_genres = genre_stats.head(10)
axes[0,0].bar(range(len(top_genres)), top_genres['ゲーム数'])
axes[0,0].set_title('ジャンル別ゲーム数 (Top 10)', fontsize=14, fontweight='bold')
axes[0,0].set_xticks(range(len(top_genres)))
axes[0,0].set_xticklabels(top_genres.index, rotation=45, ha='right')
axes[0,0].set_ylabel('ゲーム数')

# 2. インディーゲーム率（上位10ジャンル）
axes[0,1].bar(range(len(top_genres)), top_genres['インディー率'], color='orange')
axes[0,1].set_title('ジャンル別インディー率 (Top 10)', fontsize=14, fontweight='bold')
axes[0,1].set_xticks(range(len(top_genres)))
axes[0,1].set_xticklabels(top_genres.index, rotation=45, ha='right')
axes[0,1].set_ylabel('インディー率 (%)')

# 3. ジャンル別平均価格
axes[1,0].bar(range(len(top_genres)), top_genres['平均価格'], color='green')
axes[1,0].set_title('ジャンル別平均価格 (Top 10)', fontsize=14, fontweight='bold')
axes[1,0].set_xticks(range(len(top_genres)))
axes[1,0].set_xticklabels(top_genres.index, rotation=45, ha='right')
axes[1,0].set_ylabel('平均価格 ($)')

# 4. 価格帯分布
price_dist = df_processed['price_category'].value_counts()
axes[1,1].pie(price_dist.values, labels=price_dist.index, autopct='%1.1f%%')
axes[1,1].set_title('価格帯分布', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 5. インディーゲーム詳細分析

In [ ]:
# インディーゲームのみのデータフレーム
indie_df = df_processed[df_processed['is_indie'] == True].copy()

print(f"🎯 インディーゲーム詳細分析 ({len(indie_df):,}件)")
print("="*60)

# インディーゲームの価格分析
indie_paid = indie_df[indie_df['price_usd'] > 0]
if len(indie_paid) > 0:
    print(f"💰 インディーゲーム価格統計:")
    print(f"   平均価格: ${indie_paid['price_usd'].mean():.2f}")
    print(f"   中央値: ${indie_paid['price_usd'].median():.2f}")
    print(f"   標準偏差: ${indie_paid['price_usd'].std():.2f}")
    
    # 価格帯分布
    indie_price_dist = indie_df['price_category'].value_counts()
    print(f"\n📊 インディーゲーム価格帯分布:")
    for category, count in indie_price_dist.items():
        percentage = count / len(indie_df) * 100
        print(f"   {category}: {count:,}件 ({percentage:.1f}%)")

# ジャンル分析
indie_genres = indie_df['primary_genre'].value_counts().head(10)
print(f"\n🎮 インディーゲーム人気ジャンル TOP 10:")
for i, (genre, count) in enumerate(indie_genres.items(), 1):
    percentage = count / len(indie_df) * 100
    print(f"   {i:2d}. {genre}: {count:,}件 ({percentage:.1f}%)")

# プラットフォーム対応分析
print(f"\n🖥️ インディーゲームプラットフォーム対応:")
print(f"   Windows対応: {indie_df['platforms_windows'].mean()*100:.1f}%")
print(f"   Mac対応: {indie_df['platforms_mac'].mean()*100:.1f}%")
print(f"   Linux対応: {indie_df['platforms_linux'].mean()*100:.1f}%")
print(f"   平均対応数: {indie_df['platform_count'].mean():.1f}")

In [ ]:
# インディーゲームの詳細可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. インディーゲーム価格分布（ヒストグラム）
if len(indie_paid) > 0:
    axes[0,0].hist(indie_paid['price_usd'], bins=30, edgecolor='black', alpha=0.7)
    axes[0,0].set_title('インディーゲーム価格分布', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('価格 ($)')
    axes[0,0].set_ylabel('ゲーム数')
    axes[0,0].axvline(indie_paid['price_usd'].mean(), color='red', linestyle='--', label=f'平均: ${indie_paid["price_usd"].mean():.2f}')
    axes[0,0].legend()

# 2. インディーゲーム人気ジャンル
top_indie_genres = indie_genres.head(8)
axes[0,1].barh(range(len(top_indie_genres)), top_indie_genres.values)
axes[0,1].set_title('インディーゲーム人気ジャンル', fontsize=14, fontweight='bold')
axes[0,1].set_yticks(range(len(top_indie_genres)))
axes[0,1].set_yticklabels(top_indie_genres.index)
axes[0,1].set_xlabel('ゲーム数')

# 3. プラットフォーム対応比較
platform_data = {
    'Windows': [indie_df['platforms_windows'].mean()*100, 
                df_processed[df_processed['is_indie']==False]['platforms_windows'].mean()*100],
    'Mac': [indie_df['platforms_mac'].mean()*100,
            df_processed[df_processed['is_indie']==False]['platforms_mac'].mean()*100],
    'Linux': [indie_df['platforms_linux'].mean()*100,
              df_processed[df_processed['is_indie']==False]['platforms_linux'].mean()*100]
}

x = np.arange(len(platform_data))
width = 0.35

axes[1,0].bar(x - width/2, [platform_data[p][0] for p in platform_data], width, label='インディー', alpha=0.8)
axes[1,0].bar(x + width/2, [platform_data[p][1] for p in platform_data], width, label='非インディー', alpha=0.8)
axes[1,0].set_title('プラットフォーム対応率比較', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('対応率 (%)')
axes[1,0].set_xticks(x)
axes[1,0].set_xticklabels(platform_data.keys())
axes[1,0].legend()

# 4. 価格帯比較（インディー vs 非インディー）
price_comparison = pd.crosstab(df_processed['price_category'], df_processed['is_indie'], normalize='columns') * 100
price_comparison.plot(kind='bar', ax=axes[1,1], alpha=0.8)
axes[1,1].set_title('価格帯分布比較', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('割合 (%)')
axes[1,1].set_xlabel('価格帯')
axes[1,1].legend(['非インディー', 'インディー'])
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. 開発者・パブリッシャー分析

In [ ]:
# 活発な開発者・パブリッシャー分析
developer_stats = indie_df.groupby('primary_developer').agg({
    'app_id': 'count',
    'price_usd': 'mean'
}).round(2)

developer_stats.columns = ['ゲーム数', '平均価格']
developer_stats = developer_stats[developer_stats['ゲーム数'] >= 2].sort_values('ゲーム数', ascending=False)

print("🏢 活発なインディー開発者 TOP 15")
print("="*60)
print(developer_stats.head(15))

# 開発者の特徴分析
solo_developers = developer_stats[developer_stats['ゲーム数'] == 1]
prolific_developers = developer_stats[developer_stats['ゲーム数'] >= 3]

print(f"\n📊 開発者分析:")
print(f"   単発開発者: {len(solo_developers):,}人")
print(f"   多作開発者 (3+): {len(prolific_developers):,}人")
print(f"   平均作品数: {developer_stats['ゲーム数'].mean():.1f}")

if len(prolific_developers) > 0:
    print(f"\n🚀 多作開発者の特徴:")
    print(f"   平均作品数: {prolific_developers['ゲーム数'].mean():.1f}")
    print(f"   平均価格: ${prolific_developers['平均価格'].mean():.2f}")

## 7. 市場トレンドと洞察

In [ ]:
# 重要な洞察の抽出
print("🔍 Steam インディーゲーム市場 主要洞察")
print("="*80)

# 1. 市場規模と構造
print(f"📊 市場構造:")
print(f"   • インディーゲーム比率: {indie_ratio:.1f}% (Steam市場の主要セグメント)")
print(f"   • 平均価格差: インディー${indie_paid['price_usd'].mean():.2f} vs 非インディー${non_indie_paid['price_usd'].mean():.2f}")

# 2. 価格戦略の洞察
budget_indie = len(indie_df[indie_df['price_category'].isin(['Free', 'Budget ($0-5)'])])
budget_ratio = budget_indie / len(indie_df) * 100
print(f"\n💰 価格戦略:")
print(f"   • 低価格戦略: {budget_ratio:.1f}%のインディーが$5以下")
print(f"   • 価格帯集中: ${indie_paid['price_usd'].quantile(0.25):.2f} - ${indie_paid['price_usd'].quantile(0.75):.2f} (IQR)")

# 3. ジャンル特性
top_indie_genre = indie_genres.index[0]
top_genre_ratio = indie_genres.iloc[0] / len(indie_df) * 100
print(f"\n🎮 ジャンル特性:")
print(f"   • 主要ジャンル: {top_indie_genre} ({top_genre_ratio:.1f}%)")
print(f"   • ジャンル多様性: {len(indie_genres)}種類の主要ジャンル")

# 4. プラットフォーム戦略
multi_platform = len(indie_df[indie_df['platform_count'] >= 2]) / len(indie_df) * 100
print(f"\n🖥️ プラットフォーム戦略:")
print(f"   • マルチプラットフォーム率: {multi_platform:.1f}%")
print(f"   • Linux対応率: {indie_df['platforms_linux'].mean()*100:.1f}% (非インディーより高い可能性)")

# 5. 開発者エコシステム
active_developers = len(developer_stats)
avg_portfolio = developer_stats['ゲーム数'].mean()
print(f"\n👥 開発者エコシステム:")
print(f"   • 活発な開発者数: {active_developers:,}人")
print(f"   • 平均ポートフォリオ: {avg_portfolio:.1f}作品")
print(f"   • 成熟した市場: 多様な開発者が参加")

## 8. 次のステップと推奨事項

In [ ]:
print("🎯 分析結果に基づく推奨事項")
print("="*60)

print("\n📈 新規参入者への推奨:")
print(f"   • 価格設定: ${indie_paid['price_usd'].quantile(0.25):.2f} - ${indie_paid['price_usd'].median():.2f} の価格帯が主流")
print(f"   • ジャンル選択: {', '.join(indie_genres.head(3).index)} が人気")
print(f"   • プラットフォーム: Windows必須、Mac/Linux対応で差別化")

print("\n🔍 さらなる分析の方向性:")
print("   • レビューデータによる成功要因分析")
print("   • 時系列分析による市場トレンド")
print("   • 競合分析とポジショニング")
print("   • 機械学習による成功予測モデル")

print("\n💡 ビジネス洞察:")
print("   • インディーゲーム市場は成熟し、多様化が進行")
print("   • 低価格戦略が主流だが、品質で差別化可能")
print("   • マルチプラットフォーム対応が競争優位に")
print("   • ニッチジャンルでの専門化も有効戦略")

print("\n✅ この分析ノートブックで明らかになったこと:")
print(f"   ✓ {len(df_processed):,}件のゲームデータから市場構造を解明")
print(f"   ✓ インディーゲームの価格・ジャンル・プラットフォーム戦略を分析")
print(f"   ✓ 開発者エコシステムの特徴を把握")
print(f"   ✓ 新規参入と競争戦略の基礎データを提供")